## Notebook for ACL workflow normality testing

### <span style='color:Blue ; background :yellow;'> Python Packages and workflow parameters </span>

In [9]:
import Pyro4
import os
import sys
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
from pprint import pprint

from workflow_services import allocate_inference_service,prepare_endpoints_for_inference
from pyro_objects import ACL_Pyro_Objects

Pyro4.config.SERIALIZERS_ACCEPTED.add('pickle')
Pyro4.config.SERIALIZER = "pickle"

In [2]:
#
CV_Voltage_step_E = {'Ei':0.0,'E1':0.8,'E2':0.2,'Ef':0.0}    # in volt(V)
v_probe=np.linspace(CV_Voltage_step_E['E2'], CV_Voltage_step_E['E1'],10).reshape(-1, 1) # should be aligned with scan rate range

# Pyro Client-server TCP connection
# Client is Workflow Orchestrator, Server is instrument service.

ipAddressServer='172.17.0.3'
connectionPort='5001'
with open('./endpoint03.yaml', 'r') as file:
    g_endpoint = yaml.safe_load(file)

### <span style='color:Blue ; background :yellow;'> Orchestrator Dependency modules </span>


In [3]:

def get_VI_measurement_parameters():
    """
    This function determines if other requirements are asked by the user. For example if the user wants to noise the data by replicating the collected measurements, or to get the timestamp of the collected I-V measurements at the instrument side.
    :return:
    """
    try:
        input_parameters = input("Enter IV measurement file, projected size and timestamp(default: blank) ").split(' ')
        # example
        # instrument_mService\I-V_data\Test_Ferrocene_disconnect_working_s01.txt 8000 True
        # or
        # instrument_mService\I-V_data\Test_Ferrocene_disconnect_working_s01.txt
        if len(input_parameters) == 1:
            fileName_w_path = input_parameters[0]
            size = None
            timestamp = False
        elif len(input_parameters) == 2:
            fileName_w_path = input_parameters[0]
            size = int(input_parameters[1])
            timestamp = False
        elif len(input_parameters) == 3:
            fileName_w_path = input_parameters[0]
            size = int(input_parameters[1])
            timestamp = {"true": True, "false": False}.get(input_parameters[2].strip().lower())
        else:
            raise ValueError("Invalid number of parameters and/or type")
    except ValueError as e:
        # Handle the raised ValueError
        print(f"Caught an error: {e}")

    except Exception as e:
        # Catch any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
    return fileName_w_path, size, timestamp

###################################################################################
###################################################################################
def plot_IV_profiles(I,Ewe,y_pred,i_probe_t,v_probe):

    def I_V_Plot(lst_plots,title=''):
        for indx in lst_plots:
            plt.scatter(indx[1],indx[0],label=indx[2])
            plt.xlabel('V', fontsize = 16)
            plt.ylabel('I', fontsize = 16)
            plt.yticks(fontsize = 13)
            plt.xticks(fontsize = 13)
            plt.gca().yaxis.offsetText.set_fontsize(16)
            plt.ticklabel_format(axis='y', style='sci',scilimits=(0,0))
            plt.legend(loc='best')
            plt.tight_layout()
            plt.title(title)

    plt.figure(figsize=(8,4))
    #plt.suptitle(f' CV Measurement Analysis: {save_file_name}')
    ax1 = plt.subplot(1, 2, 1)
    I_V_Plot([(I,Ewe,'Measurments')],title='I-V Plot')

    #####################################
    ax2=plt.subplot(1, 2, 2)
    I_V_Plot([(y_pred,Ewe,'Regression'),(i_probe_t,v_probe,'Regression Probing')],title='I-V Regression and Probing')
    plt.show()


### <span style='color:Blue ; background :yellow;'> Orchestrator and Instrument services tasks over Pyro </span>


#### <span style='color:Blue'> Establish Client-Server Connection </span>

In [5]:
ACL_WF=ACL_Pyro_Objects(ipAddressServer,connectionPort)

#### <span style='color:Blue'> List available I-V dataset at the instrument side  </span>

In [6]:
ACL_WF.get_IV_dataset()

instrument_mService\I-V_data\Test_Ferrocene_disconnect_counter_s01.txt: size: 39848 bytes
instrument_mService\I-V_data\Test_Ferrocene_disconnect_refrence_s01.txt: size: 36915 bytes
instrument_mService\I-V_data\Test_Ferrocene_disconnect_working_s01.txt: size: 40071 bytes
instrument_mService\I-V_data\Test_Ferrocene_normal_100KB.txt: size: 137716 bytes
instrument_mService\I-V_data\Test_Ferrocene_normal_1KB.txt: size: 1296 bytes
instrument_mService\I-V_data\Test_Ferrocene_normal_1MB.txt: size: 984761 bytes
instrument_mService\I-V_data\Test_Ferrocene_normal_s01.txt: size: 38281 bytes


#### <span style='color:Blue'> Choose which I-V profile want to test its normality (the size can be noised)  </span>


In [ ]:
fileName_w_path, size, timestamp = get_VI_measurement_parameters()

 #### <span style='color:Blue'> Transfer profile measurements from the Instrument side to the Orchestrator  </span>

In [11]:
IV_df = ACL_WF.get_IV_data(fileName_w_path, size, timestamp)
pprint(IV_df)
fileName_w_path = os.path.basename(fileName_w_path)[:-4]
I = np.array(IV_df.I).reshape(-1, 1)
Ewe = np.array(IV_df.Ewe).reshape(-1, 1)


KeyboardInterrupt: Interrupted by user

### <span style='color:Blue ; background :yellow;'> Orchestrator and Compute services over Globus </span>


#### <span style='color:Blue'> Test the endpoint if it is available, then send I, and V (Ewe) of the collected I-V profile to endpoint for the inference </span>

In [ ]:
prepare_endpoints_for_inference(g_endpoint)

print("\n##################    Inference Allocation for Testing I-V profile      #################################")
i_probe_t, y_pred, profile_class, elapsed_time = allocate_inference_service(fileName_w_path, I, Ewe, g_endpoint)
print(f" IV Profile {fileName_w_path} is \
         {('Normal') if profile_class else ('Invalid')}")

plot_IV_profiles(I,Ewe,y_pred,i_probe_t,v_probe)

### <span style='color:Blue ; background :yellow;'> Terminate the connection with Instrument service </span>

In [ ]:
ACL_WF.call_Shutdown()